# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [1]:
git_token = "YOUR_GIT_TOKEN"
short_code = "YOUR_SHORT_CODE"

# Check the current python version (It should be using Python 3.10) and update pip to the latest version.
!python --version
!python -m pip install --user --upgrade pip

# Clone MASE from your branch (the branch must already exist)
!git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# Install requirements
!python -m pip install -r ./mase/machop/requirements.txt

# Change working directory to machop
%cd ./mase/machop/

Python 3.11.7
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 495.5 kB/s eta 0:00:05
   ----- ---------------------------------- 0.3/2.1 MB 2.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.1 MB 4.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.2/2.1 MB 6.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.1 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Cloning into 'mase'...
fatal: Remote branch lab1_YOUR_SHORT_CODE not found in upstream origin


[WinError 3] 系统找不到指定的路径。: './mase/machop/'
e:\OneDrive - Imperial College London\FYP\Mase-DeepWok\docs\labs


ERROR: Could not open requirements file: [Errno 2] No such file or directory: './mase/machop/requirements.txt'
e:\Programs\miniconda\envs\py_adls\Lib\site-packages\IPython\core\magics\osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
!./ch --help

# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [2]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

In [3]:
%cd ../../machop

e:\OneDrive - Imperial College London\FYP\Mase-DeepWok\machop


e:\Programs\miniconda\envs\py_adls\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

e:\Programs\miniconda\envs\py_adls\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [5]:
batch_size = 8
model_name = "jsc-zixian"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


## Set up the model 

Here we use the previously trained `jsc-tiny` model in lab 1 as an example.

In [5]:
# If you stored your model checkpoint on Google Drive, remember to mount the drive to the current runtime in order to access it
from google.colab import drive
drive.mount('/content/drive')


KeyboardInterrupt



In [6]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "../mase_output/jsc-zixian_classification_jsc_2024-01-30/software/training_ckpts/best.ckpt"
# CHECKPOINT_PATH = "../mase_output/jsc-tiny_classification_jsc_2024-01-26/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from ../mase_output/jsc-zixian_classification_jsc_2024-01-30/software/training_ckpts/best.ckpt


# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [7]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)


## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [8]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [9]:
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

We will first run a simple graph analysis to understand the structure of the model.

In [9]:
print(str(mg.fx_graph))

graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3


In [10]:
count = 0
for node in mg.fx_graph.nodes:
    print(node.meta['mase'].module)
    print(type(node.meta['mase'].module))
    print()
    if (count >= 11): 
        break
    count += 1

None
<class 'NoneType'>

BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
<class 'torch.nn.modules.batchnorm.BatchNorm1d'>

ReLU(inplace=True)
<class 'torch.nn.modules.activation.ReLU'>

Linear(in_features=16, out_features=5, bias=True)
<class 'torch.nn.modules.linear.Linear'>

ReLU(inplace=True)
<class 'torch.nn.modules.activation.ReLU'>

None
<class 'NoneType'>



In [12]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3
Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 0, 'call_method': 0, 'call_module': 4, 'output': 1}
Layer types:
[BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Linear(in_features=16, out_features=5, bias=True), ReLU(inplace=True)]


## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [13]:


pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [14]:
mg, _ = profile_statistics_analysis_pass(mg, pass_args)

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 235.33it/s]


In [15]:
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| Node name    | Fx Node op   | Mase type           | Mase op      | Software Param                                                                          |
+==============+==============+=====================+==============+=========================================================================================+
| x            | placeholder  | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                               |
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| seq_blocks_0 | call_module  | module              | batch_norm1d | {'args': {'bias': {'stat': {}},                               

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [10]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
    "relu": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [11]:
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       6 |         0 |           6 |
| Linear          | linear       |       5 |         5 |           0 |
| ReLU            | relu         |       6 |         6 |           0 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


## Q4: Traversing mg and ori_mg

In [45]:
from chop.passes.graph.utils import get_mase_op, get_mase_type, get_node_actual_target
import pandas as pd
from tabulate import tabulate


def get_data_cfg(node, kw):
  try:
    result = node.meta['mase'].parameters['common']['args'][kw]
  except:
    result = {'type':'NA', 'precision':'NA', 'shape':'NA'}
  return result

headers = ['Node', 'MASE OP', 'Ori Data Type & Prec', 'Quan Data Type & Prec', 'Ori Module', 'Quan Module']
rows = []

for ori_n, n in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
  rows.append(
      [
          ori_n.name,
          get_mase_op(ori_n),
          get_data_cfg(ori_n, 'data_in_0')['type'] + ': ' + get_data_cfg(ori_n, 'data_in_0')['precision'].__str__(),
          get_data_cfg(n, 'data_in_0')['type'] + ': ' + get_data_cfg(n, 'data_in_0')['precision'].__str__(),
          get_node_actual_target(ori_n).__class__.__name__,
          get_node_actual_target(n).__class__.__name__
      ]
  )
print("Compare nodes:")
print("\n" + tabulate(rows, headers=headers, tablefmt="orgtbl"))
        
# for ori_n, n in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
#     print('Ori: Node=%s\tNode op=%s\ttype=%s'%(ori_n.name, get_mase_op(ori_n), get_mase_type(ori_n)))
#     print('New: Node=%s\tNode op=%s\ttype=%s'%(n.name, get_mase_op(n), get_mase_type(n)))
#     try:
#         # print(n.meta['mase'].parameters['common']['results']['data_out_0']['precision'])
#         print(n.meta['mase'].parameters['common']['args']['data_in_0']['precision'])
#         print(n.meta['mase'].parameters['common']['args']['data_in_0']['type'])
#     except:
#         pass
#     # print(type(get_node_actual_target(ori_n)))
#     # print(type(get_node_actual_target(n)))
#     print(get_node_actual_target(ori_n))
#     print(get_node_actual_target(n))
#     print(get_node_actual_target(n).__class__.__name__)
#     print('---------------------')
    


Compare nodes:

| Node         | MASE OP      | Ori Data Type & Prec   | Quan Data Type & Prec   | Ori Module   | Quan Module   |
|--------------+--------------+------------------------+-------------------------+--------------+---------------|
| x            | placeholder  | NA: NA                 | NA: NA                  | str          | str           |
| seq_blocks_0 | batch_norm1d | float: [32]            | float: [32]             | BatchNorm1d  | BatchNorm1d   |
| seq_blocks_1 | relu         | float: [32]            | integer: [8, 4]         | ReLU         | ReLUInteger   |
| seq_blocks_2 | linear       | float: [32]            | integer: [8, 4]         | Linear       | LinearInteger |
| seq_blocks_3 | relu         | float: [32]            | integer: [8, 4]         | ReLU         | ReLUInteger   |
| output       | output       | NA: NA                 | NA: NA                  | str          | str           |


## Q6: Verifying Quantization

In [25]:
# Get input data
test_x = iter(data_module.val_dataloader())
xs, ys = next(test_x)

# Find quantized relu module
for ori_n, n in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
    if (get_mase_op(n) == 'relu'):
        ori_relu = get_node_actual_target(ori_n)
        relu = get_node_actual_target(n)
    if (get_mase_op(n) == 'linear'):
        ori_linear = get_node_actual_target(ori_n)
        linear = get_node_actual_target(n)
print('Ori_ReLU:', ori_relu(xs))
print('ReLU:', relu(xs))
print('Ori Linear:', ori_linear(xs))
print('Linear:', linear(xs))

Ori_ReLU: tensor([[0.0000, 0.8195, 2.0237, 2.1123, 1.9694, 2.1794, 0.0000, 0.2523, 0.0000,
         0.7871, 0.8023, 1.2439, 0.6504, 0.9804, 1.8541, 1.1823],
        [0.0000, 0.6081, 0.1573, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.2638, 0.0000, 0.0000, 0.0000, 0.0334, 0.1212],
        [1.7876, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.2838, 2.1135, 2.2838,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1767, 0.0000, 0.0000],
        [0.0000, 0.7905, 0.2666, 0.0000, 0.4394, 0.0000, 0.2284, 0.0000, 0.2284,
         0.0310, 0.2823, 0.0000, 0.2832, 0.0000, 0.3281, 0.3623],
        [0.0000, 0.4909, 0.0000, 0.0000, 0.0000, 0.0000, 0.6267, 1.2983, 0.6267,
         1.9422, 2.0859, 2.1250, 1.5999, 1.6622, 0.0000, 0.5070],
        [0.9799, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0489, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5174, 0.3022, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.



# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).